<a href="https://colab.research.google.com/github/Lesiazhou/QM2/blob/main/New_Precipitation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from netCDF4 import Dataset
import numpy as np
import pandas as pd
import requests
import scipy

#########################################################################
# inputs & parameters

# Location of URL

# Local file into which the URL is downloaded
file_download = '/Users/dianecomon/Desktop/UCL/BASC0005 QM2/Datasets/large.nc'

#
# Two other files are available, but not used
#
# Full time coverage
URL = 'https://downloads.psl.noaa.gov//Datasets/gpcc/combined/precip.comb.v2020to2019-v2020monitorafter.total.nc'


# min and max ldate, atitude and longitude to extract
min_lat = 40
max_lat = 53
min_lon = 56
max_lon = 68
min_date = '1980-01-01'
max_date = '2019-12-31'


# Where do we store the resulting CSV?
file_CSV_main = '/Users/dianecomon/Desktop/UCL/BASC0005 QM2/Datasets/precipitation_by_location.csv'
file_feather_main = '/Users/dianecomon/Desktop/UCL/BASC0005 QM2/Datasets/precipitation_by_location.feather'
file_CSV_avg = '/Users/dianecomon/Desktop/UCL/BASC0005 QM2/Datasets/precipitation_avg_locations.csv'
file_CSV_one_location = '/Users/dianecomon/Desktop/UCL/BASC0005 QM2/Datasets/precipitation_one_location.csv'

#########################################################################
# Download file

# Download PDFs

# Should run this part only once, as the download is very long
if False:
    response = requests.get(URL)
    with open(file_download, 'wb') as f:
        f.write(response.content)

# Load object
x = Dataset(file_download, 'r')
ref_date = pd.DatetimeIndex(['1800-01-01'])

# Extract all data
lat=x['lat'][:]
lon=x['lon'][:]
t = x['time'][:]
p = x['precip'][:]
t_mapped = [ref_date+pd.DateOffset(days=d) for d in t]
date = [d.strftime('%Y-%m-%d')[0] for d in t_mapped]

# Check that dimensions are congruent
if len(lat)!=p.shape[1]:
    raise ValueError('Number of longitude points mismatches')
if len(lon)!=p.shape[2]:
    raise ValueError('Number of longitude points mismatches')

#########################################################################
# Filter

# Sub-select the data for the desired range of latitude/longitude
ix_lat = np.where((lat<=max_lat)&(lat>=min_lat))[0].tolist()
ix_lon = np.where((lon<=max_lon)&(lon>=min_lon))[0].tolist()
ix_date = [i for i, d in enumerate(date) if (d>=min_date) and (d<=max_date)]
y = np.zeros((p.shape[0], len(ix_lat), len(ix_lon)))
for i_lat, v_lat in enumerate(ix_lat):
    for i_lon, v_lon in enumerate(ix_lon):
        y[:, i_lat,i_lon] = p[:, v_lat,v_lon]
select_precipitation = np.zeros((len(ix_date), y.shape[1], y.shape[2]))
for i_date, v_date in enumerate(ix_date):
    select_precipitation[i_date, :, :] = y[v_date, :, :]
select_lat = lat[ix_lat]
select_lon = lon[ix_lon]
select_date = [date[i] for i in ix_date]

if True:

    # Export full precipitations data into a Pandas
    exp_precip = pd.DataFrame(index=[i for i in range(len(select_precipitation.flatten()))] , columns=['Date','Latitude','Longitude','Precipitation'])
    i = 0
    for i_lon, lon in enumerate(select_lon):
        for i_lat, lat in enumerate(select_lat):
            for i_d, d in enumerate(select_date):
                exp_precip.loc[i, 'Date'] = select_date[i_d]
                exp_precip.loc[i, 'Latitude'] = select_lat[i_lat]
                exp_precip.loc[i, 'Longitude'] = select_lon[i_lon]
                exp_precip.loc[i, 'Precipitation'] = select_precipitation[i_d, i_lat, i_lon]
                i += 1
    exp_precip.loc[exp_precip.Precipitation<0, 'Precipitation'] = np.nan

    # Save full precipitations file to CSV
    print('Saving ', file_CSV_main)
    exp_precip.to_csv(file_CSV_main)
    exp_precip.to_feather(file_feather_main)
    print('Done')

    # Save precipitations CSV for a single location
    sel_lat = 45.875
    sel_lon =  59.375
    exp_precip[(exp_precip.Latitude==sel_lat)&(exp_precip.Longitude==sel_lon)].to_csv(file_CSV_one_location)

    # Save precipitations CSV for the average of locations
    exp_precip.groupby(by='Date')[['Precipitation']].mean().to_csv(file_CSV_avg)

    if False:
        a = exp_precip[(exp_precip.Latitude==sel_lat)&(exp_precip.Longitude==sel_lon)].copy()
        a['Year'] = [int(s[:4]) for s in a.Date]
        a['Month'] = [int(s[5:7]) for s in a.Date]
        b = a.groupby(by=['Year','Latitude','Longitude'])[['Precipitation']].sum()
        c = a.groupby(by=['Month','Latitude','Longitude'])[['Precipitation']].mean()
        d = a.pivot(values='Precipitation', columns='Month',index='Year')

Saving large.nc to large.nc


In [ ]:

!pip install netCDF4

from netCDF4 import Dataset
import numpy as np
import pandas as pd
import requests
import scipy


# inputs & parameters

# Location of URL
URL = 'https://downloads.psl.noaa.gov//Datasets/gpcc/full_v2020/precip.mon.total.0.25x0.25.v2020.nc'

# Local file into which the URL is downloaded
file_download = '/Users/zhoutao/Downloads/local_file.nc'



In [ ]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-



# Two other files are available, but not used
#
# Full time coverage
# https://downloads.psl.noaa.gov//Datasets/gpcc/combined/precip.comb.v2020to2019-v2020monitorafter.total.nc


# min and max ldate, atitude and longitude to extract
min_lat = 44
max_lat = 49
min_lon = 58
max_lon = 63
min_date = '1980-01-01'
max_date = '2024-01-01'

file_NDVI = '/Users/zhoutao/Desktop/UCL/BASC0005 QM2/Datasets/ndvi.csv'
file_NDVI_with_precipitation = '/Users/dianecomon/Desktop/UCL/BASC0005 QM2/Datasets/NDVI_with_precipitation.csv'


# Where do we store the resulting CSV?
file_CSV_main = '/Users/dianecomon/Desktop/UCL/BASC0005 QM2/Datasets/precipitation_by_location.csv'
file_CSV_avg = '/Users/dianecomon/Desktop/UCL/BASC0005 QM2/Datasets/precipitation_avg_locations.csv'
file_CSV_one_location = '/Users/dianecomon/Desktop/UCL/BASC0005 QM2/Datasets/precipitation_one_location.csv'

#########################################################################
# Download file

# Download PDFs

# Should run this part only once, as the download is very long
if False:
 response = requests.get(URL)
 with open(file_download, 'wb') as f:
     f.write(response.content)

# Load object
x = Dataset(file_download, 'r')
ref_date = pd.DatetimeIndex(['1800-01-01'])

# Extract all data
lat=x['lat'][:]
lon=x['lon'][:]
t = x['time'][:]
p = x['precip'][:]
t_mapped = [ref_date+pd.DateOffset(days=d) for d in t]
date = [d.strftime('%Y-%m-%d')[0] for d in t_mapped]

# Check that dimensions are congruent
if len(lat)!=p.shape[1]:
 raise ValueError('Number of longitude points mismatches')
if len(lon)!=p.shape[2]:
 raise ValueError('Number of longitude points mismatches')

#########################################################################
# Filter

# Sub-select the data for the desired range of latitude/longitude
ix_lat = np.where((lat<=max_lat)&(lat>=min_lat))[0].tolist()
ix_lon = np.where((lat<=max_lon)&(lat>=min_lon))[0].tolist()
ix_date = [i for i, d in enumerate(date) if (d>=min_date) and (d<=max_date)]
y = np.zeros((p.shape[0], len(ix_lat), len(ix_lon)))
for i_lat, v_lat in enumerate(ix_lat):
 for i_lon, v_lon in enumerate(ix_lon):
     y[:, i_lat,i_lon] = p[:, v_lat,v_lon]
select_precipitation = np.zeros((len(ix_date), y.shape[1], y.shape[2]))
for i_date, v_date in enumerate(ix_date):
 select_precipitation[i_date, :, :] = y[v_date, :, :]
select_lat = lat[ix_lat]
select_lon = lat[ix_lon]
select_date = [date[i] for i in ix_date]

#########################################################################
# Combine with NDVI dataset

# Load NDVI file
NDVI = pd.read_csv(file_NDVI)

# Add a column for month+year
NDVI['MonthYear'] = [s[:7] for s in NDVI.Date]

# Get the list of month ends (we do not want to calculate mapping for each day)
NDVI_dates = NDVI[['Date','MonthYear']].sort_values(by='Date').drop_duplicates().copy()
NDVI_dates = NDVI_dates.groupby(by='MonthYear')['Date'].max()

# Get the list of latitude-longitude pairs
NDVI_LL = NDVI[['Latitude','Longitude']].drop_duplicates().copy()

# Sub-sample NDVI to monthly data
NDVI_monthly = NDVI[NDVI.Date.isin(NDVI_dates.values)].copy()

# Add a placeholder for precipitation in the NDVI table
NDVI_monthly['Precipitation'] = np.nan

# For each month and each longitude/latitude in NDVI, calculate the
# 2D-interpolated precipitation
for month_NDVI in NDVI_dates.index:
 print(month_NDVI)
 ix_date_precip = select_date.index(month_NDVI+'-01')
 ix_NDVI_obs = NDVI_monthly[NDVI_monthly.MonthYear==month_NDVI].index
 f = scipy.interpolate.RectBivariateSpline(select_lat[::-1], select_lon[::-1], select_precipitation[ix_date_precip, ::-1, ::-1], kx=1, ky=1)
 for i in ix_NDVI_obs:
     interp_val = f(NDVI_monthly.loc[i, 'Latitude'], NDVI_monthly.loc[i, 'Longitude'])
     NDVI_monthly.loc[i, 'Precipitation'] = interp_val



#########################################################################
# Export the data to CSV files

# Export the monthly NDVI with precipitations data
NDVI_monthly.to_csv(file_NDVI_with_precipitation)

# Export full precipitations data into a Pandas
exp_precip = pd.DataFrame(index=[i for i in range(len(select_precipitation.flatten()))] , columns=['Date','Latitude','Longitude','Precipitation'])
i = 0
for i_lon, lon in enumerate(select_lon):
 for i_lat, lat in enumerate(select_lat):
     for i_d, d in enumerate(select_date):
         exp_precip.loc[i, 'Date'] = select_date[i_d]
         exp_precip.loc[i, 'Latitude'] = select_lat[i_lat]
         exp_precip.loc[i, 'Longitude'] = select_lon[i_lon]
         exp_precip.loc[i, 'Precipitation'] = select_precipitation[i_d, i_lat, i_lon]
         i += 1
exp_precip.loc[exp_precip.Precipitation<0, 'Precipitation'] = np.nan

# Save full precipitations file to CSV
exp_precip.to_csv(file_CSV_main)

# Save precipitations CSV for a single location
sel_lat = 45.875
sel_lon =  59.375
exp_precip[(exp_precip.Latitude==sel_lat)&(exp_precip.Longitude==sel_lon)].to_csv(file_CSV_one_location)

# Save precipitations CSV for the average of locations
exp_precip.groupby(by='Date')[['Precipitation']].mean().to_csv(file_CSV_avg)

if False:
 a = exp_precip[(exp_precip.Latitude==sel_lat)&(exp_precip.Longitude==sel_lon)].copy()
 a['Year'] = [int(s[:4]) for s in a.Date]
 a['Month'] = [int(s[5:7]) for s in a.Date]
 b = a.groupby(by=['Year','Latitude','Longitude'])[['Precipitation']].sum()
 c = a.groupby(by=['Month','Latitude','Longitude'])[['Precipitation']].mean()
 d = a.pivot(values='Precipitation', columns='Month',index='Year')

FileNotFoundError: [Errno 2] No such file or directory: '/Users/zhoutao/Downloads/local_file.nc'